# Using Examon with Apache Spark
This example shows how to set the examon-client to execute distributed queries and analytics with Apache Spark https://spark.apache.org/.
The example covers both the JSON and the ExamonQL query interfaces.

In [1]:
import os
from examon.examon import Examon, ExamonQL
from examon.executors import ExSpark

import pyspark  #v3.1.2
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext, SparkSession

#### Configure Spark

In [2]:
# set the spark configuration
conf = SparkConf()
conf.setAppName('ExamonClientTestApp')
conf.setMaster('local[*]')

# create the sparkcontext and sparksession objects 
sc = SparkContext.getOrCreate(conf=conf)
spark = SparkSession(sc)  # needed for toDF

In [4]:
spark

#### Configure the Examon client

In [5]:
KAIROSDB_SERVER = 'examon.cineca.it'
KAIROSDB_PORT = '3000'
USER = ''
PWD = ''
ex = Examon(KAIROSDB_SERVER, port=KAIROSDB_PORT, user=USER, password=PWD, verbose=False, proxy=True)


### 1) Query in Spark using the native db interface (JSON)

In [6]:
# query some data
tstart = "12-06-2019 08:09:00"   
tstop = "19-06-2019 19:09:00"
metrics = ['Sys_Power']
tags = {'node': ['r162c16s02']}
groupby = {'name':'tag', 'tags':['node', 'plugin', 'chnl', 'cluster', 'part', 'health', 'units', 'org', 'type']}
aggrby = None

# create the spark executor, initialized with the examon-client data
sp = ExSpark(ex)

# define a Spark RDD with the content of the query data
rdd = sp \
    .spark(sc) \
    .query(tstart, tstop, metrics, tags, groupby=groupby, aggrby=aggrby, time_zone='Europe/Rome', batch_size=30*60*1000) \
    .to_rdd(npartitions=8)

Query(tstart='12-06-2019 08:09:00', tstop='19-06-2019 19:09:00', metrics=['Sys_Power'], tags={'node': ['r162c16s02']}, groupby=[{'name': 'tag', 'tags': ['node', 'plugin', 'chnl', 'cluster', 'part', 'health', 'units', 'org', 'type']}], aggrby=None, limit=None, time_zone='Europe/Rome')


Please note: The original query is sliced in chunks over the total time range. Their size is defined by the *batch_size* parameter (milliseconds) of the *query()* method. So each Spark worker will query an amount of data defined by the number of chunks per partition. The number of partitions of the resulting RDD can be defined by the *npartitions* parameter of the *to_rdd()* method. If you are not sure, you can leave *npartitions=None* (default) and it will be handled by Spark as usual.

In [7]:
# Trigger the query executing an action (take) on the RDD
rdd.take(10)

[Row(timestamp=1560319740028, value=170, name='Sys_Power', node='r162c16s02', plugin='confluent_pub', chnl='data', cluster='marconi', part='skylake', health='ok', units='W', org='cineca', type='Current'),
 Row(timestamp=1560319800032, value=110, name='Sys_Power', node='r162c16s02', plugin='confluent_pub', chnl='data', cluster='marconi', part='skylake', health='ok', units='W', org='cineca', type='Current'),
 Row(timestamp=1560319860028, value=340, name='Sys_Power', node='r162c16s02', plugin='confluent_pub', chnl='data', cluster='marconi', part='skylake', health='ok', units='W', org='cineca', type='Current'),
 Row(timestamp=1560319920034, value=340, name='Sys_Power', node='r162c16s02', plugin='confluent_pub', chnl='data', cluster='marconi', part='skylake', health='ok', units='W', org='cineca', type='Current'),
 Row(timestamp=1560319980026, value=340, name='Sys_Power', node='r162c16s02', plugin='confluent_pub', chnl='data', cluster='marconi', part='skylake', health='ok', units='W', org='c

In [8]:
# cretae a Spark dataframe from the RDD
df = rdd.toDF()
df.show(10)

+-------------+-----+---------+----------+-------------+----+-------+-------+------+-----+------+-------+
|    timestamp|value|     name|      node|       plugin|chnl|cluster|   part|health|units|   org|   type|
+-------------+-----+---------+----------+-------------+----+-------+-------+------+-----+------+-------+
|1560319740028|  170|Sys_Power|r162c16s02|confluent_pub|data|marconi|skylake|    ok|    W|cineca|Current|
|1560319800032|  110|Sys_Power|r162c16s02|confluent_pub|data|marconi|skylake|    ok|    W|cineca|Current|
|1560319860028|  340|Sys_Power|r162c16s02|confluent_pub|data|marconi|skylake|    ok|    W|cineca|Current|
|1560319920034|  340|Sys_Power|r162c16s02|confluent_pub|data|marconi|skylake|    ok|    W|cineca|Current|
|1560319980026|  340|Sys_Power|r162c16s02|confluent_pub|data|marconi|skylake|    ok|    W|cineca|Current|
|1560320040030|  350|Sys_Power|r162c16s02|confluent_pub|data|marconi|skylake|    ok|    W|cineca|Current|
|1560320100027|  350|Sys_Power|r162c16s02|conf

In [9]:
# show the inferred column data types of the spark dataframe
df.dtypes

[('timestamp', 'bigint'),
 ('value', 'bigint'),
 ('name', 'string'),
 ('node', 'string'),
 ('plugin', 'string'),
 ('chnl', 'string'),
 ('cluster', 'string'),
 ('part', 'string'),
 ('health', 'string'),
 ('units', 'string'),
 ('org', 'string'),
 ('type', 'string')]

### 2) Query in Spark using the ExamonQL interface
Here we do the same query but showing how the ExamonQL interface can be used with the Spark executor

In [10]:
# create the ExamonQl object
sq = ExamonQL(ex)

# do the query as usual but get only the query parameters using 'get_query()'
query = sq.SELECT('*') \
    .FROM('Sys_Power') \
    .WHERE(node='r162c16s02') \
    .TSTART('12-06-2019 08:09:00') \
    .TSTOP('19-06-2019 19:09:00') \
    .get_query()

In [11]:
# pass the query parameters to the spark executor 

# create the spark executor, initialized with the examon-client data
sp = ExSpark(ex)

# define the RDD passing the query parameters (query)
rdd = sp \
      .spark(sc) \
      .query(*query, batch_size=30*60*1000) \
      .to_rdd(npartitions=8)

Query(tstart='12-06-2019 08:09:00', tstop='19-06-2019 19:09:00', metrics=['Sys_Power'], tags={'node': ['r162c16s02']}, groupby=[{'name': 'tag', 'tags': ['chnl', 'cluster', 'health', 'node', 'org', 'part', 'plugin', 'type', 'units']}], aggrby=None, limit=None, time_zone='Europe/Rome')


In [12]:
# Trigger the query executing an action (take) on the RDD
rdd.take(10)

[Row(timestamp=1560319740028, value=170, name='Sys_Power', chnl='data', cluster='marconi', health='ok', node='r162c16s02', org='cineca', part='skylake', plugin='confluent_pub', type='Current', units='W'),
 Row(timestamp=1560319800032, value=110, name='Sys_Power', chnl='data', cluster='marconi', health='ok', node='r162c16s02', org='cineca', part='skylake', plugin='confluent_pub', type='Current', units='W'),
 Row(timestamp=1560319860028, value=340, name='Sys_Power', chnl='data', cluster='marconi', health='ok', node='r162c16s02', org='cineca', part='skylake', plugin='confluent_pub', type='Current', units='W'),
 Row(timestamp=1560319920034, value=340, name='Sys_Power', chnl='data', cluster='marconi', health='ok', node='r162c16s02', org='cineca', part='skylake', plugin='confluent_pub', type='Current', units='W'),
 Row(timestamp=1560319980026, value=340, name='Sys_Power', chnl='data', cluster='marconi', health='ok', node='r162c16s02', org='cineca', part='skylake', plugin='confluent_pub', typ

In [13]:
# cretae a Spark dataframe from the RDD
df = rdd.toDF()
df.show(10)

+-------------+-----+---------+----+-------+------+----------+------+-------+-------------+-------+-----+
|    timestamp|value|     name|chnl|cluster|health|      node|   org|   part|       plugin|   type|units|
+-------------+-----+---------+----+-------+------+----------+------+-------+-------------+-------+-----+
|1560319740028|  170|Sys_Power|data|marconi|    ok|r162c16s02|cineca|skylake|confluent_pub|Current|    W|
|1560319800032|  110|Sys_Power|data|marconi|    ok|r162c16s02|cineca|skylake|confluent_pub|Current|    W|
|1560319860028|  340|Sys_Power|data|marconi|    ok|r162c16s02|cineca|skylake|confluent_pub|Current|    W|
|1560319920034|  340|Sys_Power|data|marconi|    ok|r162c16s02|cineca|skylake|confluent_pub|Current|    W|
|1560319980026|  340|Sys_Power|data|marconi|    ok|r162c16s02|cineca|skylake|confluent_pub|Current|    W|
|1560320040030|  350|Sys_Power|data|marconi|    ok|r162c16s02|cineca|skylake|confluent_pub|Current|    W|
|1560320100027|  350|Sys_Power|data|marconi|  

In [14]:
# show the inferred column data types of the spark dataframe
df.dtypes

[('timestamp', 'bigint'),
 ('value', 'bigint'),
 ('name', 'string'),
 ('chnl', 'string'),
 ('cluster', 'string'),
 ('health', 'string'),
 ('node', 'string'),
 ('org', 'string'),
 ('part', 'string'),
 ('plugin', 'string'),
 ('type', 'string'),
 ('units', 'string')]